In [ ]:
import dask.dataframe as dd
from coiled import Cluster
from distributed import Client

In [ ]:
cluster = Cluster(
    name="hendrik-pycon-demo", 
    n_workers=20, 
    worker_vm_types="m6i.large", 
    scheduler_vm_types=["m6i.large"], 
    worker_disk_size=15, # Limiting disk to blow up Dask
    shutdown_on_close=False,
    scheduler_options={"idle_timeout": "24 hours"}
)
client = Client(cluster)

In [ ]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/h2o-benchmark/pyarrow_strings/N_1e10_K_1e2/*.parquet", 
    engine="pyarrow", 
    storage_options={"anon": True}
)
ddf = ddf[["id4", "id5", "v3"]]
(
    ddf.groupby(["id4", "id5"], dropna=False, observed=True)
    .agg({"v3": ["median", "std"]}, shuffle="p2p")
    .compute()
)
medians = ddf.compute()